In [ ]:
!pip install opencv-python ultralytics numpy sqlite3

In [ ]:
pip install opencv-python ultralytics numpy

In [ ]:
pip install mysql-connector-python


In [ ]:
import cv2
import sqlite3
from datetime import datetime
from ultralytics import YOLO

# Load YOLOv5 model (you can also use yolov8)
model = YOLO('yolov5s.pt')  # download will happen automatically

# Connect to SQLite (for MySQL use mysql.connector)
conn = sqlite3.connect('people_tracking.db')
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS people_visits (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    count INTEGER
)
""")
conn.commit()

# Choose your video source
video_source = 0  # use 0 for webcam or 'http://ip-camera-url' for IP cam
cap = cv2.VideoCapture(video_source)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, stream=True)
    people_count = 0

    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            if model.names[cls] == "person":
                people_count += 1
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Show people count
    cv2.putText(frame, f"People Count: {people_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Save to database
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute("INSERT INTO people_visits (timestamp, count) VALUES (?, ?)", (timestamp, people_count))
    conn.commit()

    cv2.imshow("People Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
conn.close()
cv2.destroyAllWindows()

       